In [1]:

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)
        
        for page in PDFPage.get_pages(file, check_extractable=True):
            page_interpreter.process_page(page)
            text += fake_file_handle.getvalue()
        
        converter.close()
        fake_file_handle.close()
    
    return text
# Function to divide text into chunks of 512 tokens
def divide_text_into_chunks(text, max_chunk_size=512):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        # print(sentence)
        # print("##########")
        tokenized_sentence = sentence.split()
        if len(current_chunk) + len(tokenized_sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# text_splitter_1 = TokenTextSplitter(chunk_size=100,chunk_overlap = 2)
# Example usage
pdf_path = './15031-4983-FullBook.pdf'
text = extract_text_from_pdf(pdf_path)
chunks = divide_text_into_chunks(text)
def check_grammar(sentences, model, tokenizer):
    for sentence in sentences:
        sentence_label = get_first_sentence_prediction(model, tokenizer, [sentence])
        if sentence_label == 0:  # If the sentence is grammatically incorrect
            return 0
    return 1
label_mapping = {0: "incorrect", 1: "correct"}
prompt_line = f"Count the grammatical error in NLP and give me only number for the following text: "
grammatical_error_count = []
for idx, chunk in enumerate(chunks[0:15]):
    print(chunk)
    sentences = sent_tokenize(chunk)
    chunk_label = check_grammar(sentences, model, tokenizer)
    if chunk_label == 0:
        input_text = prompt_line + chunk
        conversation_list, response = chat_with_bot(input_text)
        print("Conversation List:", conversation_list)
        print("Response:", response)
        print()
        # Extract the numeric value using regular expressions
        error_count = re.findall(r'\d+', response.split("\n")[0])
        if error_count:
            # Convert the extracted value to an integer
            error_count = int(error_count[0])
        else:
            # If a numeric value is not found, try to convert words to numbers
            try:
                error_count = w2n.word_to_num(response.split("\n")[0])
            except ValueError:
                error_count = 0
        
        print("error_count:", error_count)
        grammatical_error_count.append(error_count)
        
    print(f"Chunk {idx+1} - Sentence Label: {label_mapping[chunk_label]}")
print("Grammer",grammatical_error_count,len(grammatical_error_count))     
        
        

FileNotFoundError: [Errno 2] No such file or directory: './15031-4983-FullBook.pdf'